In [1]:
import os
os.chdir('/workspace/objectdet/')

In [2]:
from mmengine.config import Config

cfg = Config.fromfile('mmdetection/configs/detr/detr_r50_8xb2-150e_coco.py')

In [3]:
cfg.model.bbox_head.num_classes = 1
cfg.data_root = "datasets/Box-1/"

metainfo = {
    'classes': ('Box', ),
    'palette': [
        (220, 20, 60),
    ]
}

cfg.train_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='train/_annotations.coco.json',
        data_prefix=dict(img='train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=cfg.train_pipeline,
        backend_args=cfg.backend_args))
cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='valid/_annotations.coco.json',
        data_prefix=dict(img='valid/'),
        test_mode=True,
        pipeline=cfg.test_pipeline,
        backend_args=cfg.backend_args))

cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'valid/_annotations.coco.json',
    metric='bbox',
    format_only=False,
    backend_args=cfg.backend_args)


# inference on test dataset and
# format the output results for submission.

cfg.test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file=cfg.data_root + 'test/_annotations.coco.json',
        data_prefix=dict(img='test/'),
        test_mode=True,
        pipeline=cfg.test_pipeline))

cfg.test_evaluator = dict(
    type='CocoMetric',
    metric='bbox',
    format_only=True,
    ann_file=cfg.data_root + 'test/_annotations.coco.json',
    outfile_prefix='./work_dirs/detr_r50_8xb2-150e_coco/test')

cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=12, val_interval=1)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.00001, weight_decay=0.0001),
    clip_grad=dict(max_norm=0.1, norm_type=2),
    paramwise_cfg=dict(
        custom_keys={'backbone': dict(lr_mult=0.1, decay_mult=1.0)}))

cfg.auto_scale_lr = dict(base_batch_size=2)
cfg.work_dir = './work_dirs/detr_r50_8xb2-150e_coco/2'

cfg.default_hooks = dict(
                checkpoint=dict(
                    type='CheckpointHook', 
                    interval=3,
                    max_keep_ckpts=2,  # only keep latest 2 checkpoints
                    save_best='auto'
                    ),
                sampler_seed=dict(type='DistSamplerSeedHook'),
                param_scheduler=dict(type='ParamSchedulerHook'),
                logger=dict(type='LoggerHook', interval=50),
                timer=dict(type='IterTimerHook'),
                visualization=dict(type='DetVisualizationHook'),
                )

# cfg.vis_backends = dict(type='LocalVisBackend')


In [4]:
from mmengine.runner import Runner

model = Runner.from_cfg(cfg)
model.train()

03/15 11:23:40 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1294087298
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencod

/workspace/objectdet/.venv/lib/python3.10/site-packages/mmdet/models/losses/cross_entropy_loss.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  class_weight = cls_score.new_tensor(


03/15 11:23:59 - mmengine - INFO - Epoch(train)  [1][ 50/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:25:14  time: 0.2768  data_time: 0.0051  memory: 1920  grad_norm: 429.6401  loss: 43.2107  loss_cls: 0.6133  loss_bbox: 4.2574  loss_iou: 2.2258  d0.loss_cls: 0.5780  d0.loss_bbox: 4.5310  d0.loss_iou: 2.1607  d1.loss_cls: 0.6125  d1.loss_bbox: 4.4786  d1.loss_iou: 2.1664  d2.loss_cls: 0.6148  d2.loss_bbox: 4.4453  d2.loss_iou: 2.1963  d3.loss_cls: 0.6043  d3.loss_bbox: 4.3421  d3.loss_iou: 2.1875  d4.loss_cls: 0.6301  d4.loss_bbox: 4.3329  d4.loss_iou: 2.2336
03/15 11:24:11 - mmengine - INFO - Epoch(train)  [1][100/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:23:24  time: 0.2415  data_time: 0.0038  memory: 1789  grad_norm: 369.0739  loss: 37.2732  loss_cls: 0.5740  loss_bbox: 3.3775  loss_iou: 2.2490  d0.loss_cls: 0.5577  d0.loss_bbox: 3.5508  d0.loss_iou: 2.2036  d1.loss_cls: 0.5696  d1.loss_bbox: 3.4034  d1.loss_iou: 2.2531  d2.loss_cls: 0.5697  d2.loss_bbox: 3.3722  d2.loss_

/workspace/objectdet/.venv/lib/python3.10/site-packages/mmdet/models/losses/cross_entropy_loss.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  class_weight = cls_score.new_tensor(


03/15 11:25:42 - mmengine - INFO - Epoch(train)  [2][ 50/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:17:47  time: 0.1893  data_time: 0.0037  memory: 1512  grad_norm: 708.8213  loss: 30.4291  loss_cls: 0.5405  loss_bbox: 2.2503  loss_iou: 2.2689  d0.loss_cls: 0.5164  d0.loss_bbox: 2.3091  d0.loss_iou: 2.2885  d1.loss_cls: 0.5348  d1.loss_bbox: 2.2610  d1.loss_iou: 2.2697  d2.loss_cls: 0.5421  d2.loss_bbox: 2.2473  d2.loss_iou: 2.2848  d3.loss_cls: 0.5372  d3.loss_bbox: 2.2626  d3.loss_iou: 2.2617  d4.loss_cls: 0.5410  d4.loss_bbox: 2.2548  d4.loss_iou: 2.2585
03/15 11:25:52 - mmengine - INFO - Epoch(train)  [2][100/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:17:26  time: 0.1885  data_time: 0.0035  memory: 1516  grad_norm: 776.8064  loss: 28.8642  loss_cls: 0.5485  loss_bbox: 1.8468  loss_iou: 2.3993  d0.loss_cls: 0.5347  d0.loss_bbox: 1.9045  d0.loss_iou: 2.4178  d1.loss_cls: 0.5444  d1.loss_bbox: 1.8399  d1.loss_iou: 2.3792  d2.loss_cls: 0.5544  d2.loss_bbox: 1.8396  d2.loss_

...

/workspace/objectdet/.venv/lib/python3.10/site-packages/mmdet/models/losses/cross_entropy_loss.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  class_weight = cls_score.new_tensor(


03/15 11:30:17 - mmengine - INFO - Epoch(train)  [5][ 50/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:11:27  time: 0.2043  data_time: 0.0039  memory: 1820  grad_norm: 2221.4347  loss: 16.5920  loss_cls: 0.5240  loss_bbox: 0.9529  loss_iou: 1.2423  d0.loss_cls: 0.4981  d0.loss_bbox: 1.0984  d0.loss_iou: 1.3916  d1.loss_cls: 0.5107  d1.loss_bbox: 1.0386  d1.loss_iou: 1.3098  d2.loss_cls: 0.5017  d2.loss_bbox: 0.9758  d2.loss_iou: 1.2462  d3.loss_cls: 0.5160  d3.loss_bbox: 0.9223  d3.loss_iou: 1.2041  d4.loss_cls: 0.5216  d4.loss_bbox: 0.9275  d4.loss_iou: 1.2104
03/15 11:30:26 - mmengine - INFO - Epoch(train)  [5][100/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:11:16  time: 0.1783  data_time: 0.0035  memory: 1510  grad_norm: 2503.0002  loss: 16.7771  loss_cls: 0.5450  loss_bbox: 0.9158  loss_iou: 1.2831  d0.loss_cls: 0.5028  d0.loss_bbox: 1.0474  d0.loss_iou: 1.4260  d1.loss_cls: 0.5213  d1.loss_bbox: 0.9597  d1.loss_iou: 1.3032  d2.loss_cls: 0.5279  d2.loss_bbox: 0.9382  d2.los

/workspace/objectdet/.venv/lib/python3.10/site-packages/mmdet/models/losses/cross_entropy_loss.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  class_weight = cls_score.new_tensor(


03/15 11:33:15 - mmengine - INFO - Epoch(train)  [7][ 50/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:08:21  time: 0.1791  data_time: 0.0038  memory: 1510  grad_norm: 2097.6818  loss: 15.7227  loss_cls: 0.5473  loss_bbox: 0.8903  loss_iou: 1.1694  d0.loss_cls: 0.5102  d0.loss_bbox: 0.9427  d0.loss_iou: 1.2634  d1.loss_cls: 0.5277  d1.loss_bbox: 0.8951  d1.loss_iou: 1.2076  d2.loss_cls: 0.5370  d2.loss_bbox: 0.8869  d2.loss_iou: 1.2083  d3.loss_cls: 0.5437  d3.loss_bbox: 0.8513  d3.loss_iou: 1.1653  d4.loss_cls: 0.5511  d4.loss_bbox: 0.8721  d4.loss_iou: 1.1533
03/15 11:33:24 - mmengine - INFO - Epoch(train)  [7][100/460]  base_lr: 1.0000e-05 lr: 1.0000e-05  eta: 0:08:11  time: 0.1784  data_time: 0.0036  memory: 1629  grad_norm: 2503.4231  loss: 15.9669  loss_cls: 0.5475  loss_bbox: 0.8660  loss_iou: 1.2077  d0.loss_cls: 0.5243  d0.loss_bbox: 0.9313  d0.loss_iou: 1.3817  d1.loss_cls: 0.5247  d1.loss_bbox: 0.8790  d1.loss_iou: 1.2527  d2.loss_cls: 0.5431  d2.loss_bbox: 0.8548  d2.los

KeyboardInterrupt: 